In [1]:
# For wide monitor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
# 살인, 살해 키워드 정보 따로 수집 -> 컬럼수 1로 표기
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook, tqdm
import time
import re
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from datetime import timedelta, date

In [9]:
import requests
import json

In [11]:
# 댓글 정보 가져오기
import requests
import json
from bs4 import BeautifulSoup


def get_comments(news_url):
    base_url = ''.join(['https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&',
                    'templateId=view_politics&pool=cbox5&lang=ko&country=KR&objectId=news',
                    '{}%2C{}&pageSize={}&page={}&sort={}&initialize=true&useAltSort=true&indexSize=10'])
    
    (oid, aid) = _parse_oid_aid(news_url)    
    if oid == None or aid == None:
        return []
    
    n_comments = _n_comments(base_url.format(oid, aid, 10, 1, 'favorite'), news_url)
    max_page = round(n_comments/100 + 0.5)
    if max_page <= 0:
        return []
    
    comments = []
    headers = {'Referer':news_url}
    for page in range(1, max_page + 1):
        url = base_url.format(oid, aid, 100, page, 'favorite')
        response = _get_response(url, {'Referer':news_url})
        for comment_json in response.get('result', {}).get('commentList', []):
            try:
                comments.append(_parse_comment(comment_json))
            except:
                continue
    return comments
    
def _get_response(url, headers=None):
    try:
        if not headers:
            r = requests.get(url)
        else:
            r = requests.get(url, headers=headers)
        html = r.text
        html = html[10:-2]
        response = json.loads(html)
        return response
    except:
        return {}

def _parse_oid_aid(news_url):
    parts = news_url.split('?')[-1].split('&')
    (oid, aid) = (None, None)
    for part in parts:
        if 'oid=' in part:
            oid = part[4:]
        if 'aid=' in part:
            aid = part[4:]
    return oid, aid

def _n_comments(url, news_url):
    response = _get_response(url, {'Referer':news_url})
    n_comments = response.get('result', {}).get('count', {}).get('comment', 0)
    return n_comments

def _parse_comment(comment_json):
    antipathy_count = comment_json['antipathyCount']
    sympathy_count = comment_json['sympathyCount']
    comment_no = comment_json['commentNo']
    contents = comment_json['contents'].replace('\t', ' ').replace('\r', ' ').replace('\n', ' ')
    reg_time = comment_json['regTime']
    user_id_no = comment_json['userIdNo']
    return (comment_no, user_id_no, contents, reg_time, sympathy_count, antipathy_count)

In [9]:
querys = ['조현병', '정신분열+"정신분열"','정신분열+"정신분열"+"사건"','조현병+"사건"','정신분열+"정신분열"+"범행"','조현병+"범행"', '정신분열+"정신분열"+"범죄"','조현병+"범죄"',
         '정신분열+"정신분열"+"살인"','조현병+"살인"', '정신분열+"정신분열"+"살해"','조현병+"살해"', '정신분열+"정신분열"+"폭행"','조현병+"폭행"',
         '정신분열+"정신분열"+"방화"','조현병+"방화"']

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(executable_path=r'C:\chromedriver.exe', chrome_options=options)

list_ = []
index = 0
for query in tqdm(querys):
    if query in ['정신분열+"정신분열"+"살인"','조현병+"살인"', '정신분열+"정신분열"+"살해"','조현병+"살해"']:
        murder = 1
    else:
        murder=0
        
    row = 0
    data = pd.DataFrame(columns=['title', 'company', 'url', 'is_murder'])
    
    url = 'https://search.naver.com/search.naver?where=news&query='+ query +'&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=2010.01.01&de=2019.05.23&docid=&nso=so%3Ar%2Cp%3Afrom20100101to20190523%2Ca%3Aall&mynews=0&refresh_start=0&related=0'
    driver.get(url)
    driver.find_element_by_id('news_popup').find_element_by_tag_name('a').click()
    
    
    id_dict = {'연합뉴스':'ca_1001','연합뉴스TV':'ca_1422','YTN':'ca_1052'}
    driver.find_element_by_id(id_dict['연합뉴스']).click()
    driver.find_element_by_id(id_dict['연합뉴스TV']).click()
    driver.find_element_by_id(id_dict['YTN']).click()
    

    driver.find_element_by_class_name('view_btn').find_element_by_tag_name('button').click()
    driver.find_element_by_class_name('sort').find_elements_by_tag_name('li')[1].click()
    
    while(True):
        titles = driver.find_elements_by_class_name('_sp_each_title')
        txt_inlines = driver.find_elements_by_class_name('txt_inline')

        for i in range(len(titles)):
            title = titles[i].get_attribute('title')
            company = txt_inlines[i].text.split(' ')[0]

            if txt_inlines[i].find_element_by_tag_name('a').text == '네이버뉴스':
                link = txt_inlines[i].find_element_by_tag_name('a').get_attribute('href')
            else:
                link = titles[i].get_attribute('href')

            list_.append({
                'title': title,
                'company': company,
                'url': link,
                'is_murder': murder
            })

            data.loc[row] = [title, company, link, murder]
            row += 1
        if driver.find_element_by_class_name('paging').find_elements_by_tag_name('a')[-1].text == '다음페이지':
            driver.find_element_by_class_name('paging').find_elements_by_tag_name('a')[-1].click()
        else:
            break

    data.to_csv('네이버뉴스_{}.csv'.format(index), index=False)
    index += 1
    
driver.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

  6%|█████                                                                           | 1/16 [09:02<2:15:37, 542.51s/it]

 12%|██████████                                                                      | 2/16 [16:18<1:59:08, 510.62s/it]

 19%|███████████████                                                                 | 3/16 [19:16<1:29:00, 410.84s/it]

 25%|████████████████████                                                            | 4/16 [23:46<1:13:42, 368.55s/it]

 31%|█████████████████████████▋                                                        | 5/16 [26:36<56:39, 309.02s/it]

 38%|██████████████████████████████▊                                                   | 6/16 [30:48<48:39, 291.98s/i

In [10]:
df_all = pd.read_csv('네이버뉴스_0.csv', encoding='UTF-8', engine='python')
# 키워드 없이 전체 기사 
temp = pd.read_csv('네이버뉴스_1.csv', encoding='UTF-8', engine='python')
df_all = pd.concat([df_all, temp])
df_all.to_csv('0622_전체_article.csv', index=False)

In [13]:
temp_data_all = df_all.drop_duplicates(['url'])
temp_data_all.reset_index(drop=True, inplace=True)
temp_data_all.to_csv('0622_전체_article_중복제거.csv', index=False)

df = pd.read_csv('네이버뉴스_2.csv', encoding='UTF-8', engine='python')
for i in range(3, 16):
    temp = pd.read_csv('네이버뉴스_{}.csv'.format(i), encoding='UTF-8', engine='python')
    df = pd.concat([df, temp])

df.to_csv('0622_article_total.csv', index=False)
naver_news = pd.read_csv('0622_article_total.csv')
temp_data = naver_news.drop_duplicates(['url'])
temp_data.reset_index(drop=True, inplace=True)

In [3]:
naver_news = pd.read_csv('0622_article_total.csv')
temp_data = naver_news.drop_duplicates(['url'])
temp_data.reset_index(drop=True, inplace=True)

In [51]:
def load_news(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    
    if soup.find('h3', {'id':'articleTitle'}):
        title = soup.find('h3', {'id':'articleTitle'}).text
        dates = []
        for date in soup.find_all('span', {'class':'t11'}):
            dates.append(date.text)
        body = soup.find('div', {'id':'articleBodyContents'}).text.replace('\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n', '')
        
        
        return title, dates[0], body
        
    elif soup.find('h2', {'class':'end_tit'}):
        title = soup.find('h2', {'class':'end_tit'}).text
        dates = []
        for date in soup.find('div', {'class':'article_info'}).text.split('\n')[1:3]:
            dates.append(re.sub('[(기사입력)(최종수정)]', '', date))
        body = soup.find('div', {'id':'articeBody'}).text
        
        return title, dates[0], body
    
    else:
        return '', '', ''

In [4]:
temp_data = naver_news.drop_duplicates(['url'])
temp_data.reset_index(drop=True, inplace=True)
temp_data.tail()

,title,company,url,is_murder
1585,자신 원룸에 불 지른 30대 조현병 환자 영장,YTN,https://news.naver.com/main/read.nhn?mode=LSD&...,0
1586,백남기 씨 빈소서 분신 시도...40대 남성 체포,YTN,https://news.naver.com/main/read.nhn?mode=LSD&...,0
1587,백남기씨 빈소서 분신 시도한 40대 남성 체포,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,0
1588,"中 톈안먼 사태 최장기수 석방…""톈안먼 재평가 요원""",연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,0
1589,中 톈안먼 시위 참가 마지막 복역수 올해 석방된다,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,0


In [53]:
import json
news = []

for i, link in enumerate(tqdm(temp_data['url'])):
    title, date, content = load_news(link)
    comment = get_comments(link)

    news.append({'title':title, 'date':date, 'content':content, 'company':temp_data['company'].loc[i], 'url':link, 'is_murder':temp_data['is_murder'].loc[i], 'comment':comment, 'comment_num':len(comment)})

100%|██████████████████████████████████████████████████████████████████████████████| 1590/1590 [16:03<00:00,  1.19it/s]


In [57]:
len(news)

1589

In [59]:
for n in range(len(news)):
    if news[n]['date']=='':
        print(news[n])

In [60]:
import json

with open(r'C:\Users\ehdry\Downloads\네이버뉴스_조현병.txt','r') as f:
    list_ = json.load(f)
    list_.extend(news)

In [69]:
#news = list_

for n in range(len(news)):
    if news[n]['date']=='':
        del news[n]

In [70]:
len(news)

3286

In [75]:
# json dump 저장
import json

with open(r'C:\Users\ehdry\Downloads\네이버뉴스_조현병_final_20190801.txt','w') as f:
    json.dump(news, f, default=str)
    #list_.extend(news)

In [76]:
with open(r'C:\Users\ehdry\Downloads\네이버뉴스_조현병_final_20190801.txt','r') as f:
    json_file = json.load(f)

In [80]:
json_file[0].keys()

dict_keys(['title', 'date', 'content', 'company', 'url', 'is_murder', 'comment', 'comment_num', 'year', 'month'])

In [81]:
import csv

with open('조현병_20190801.csv', 'w', encoding='utf-8', newline='') as f:
    fieldnames = ['title', 'date', 'content', 'company', 'url', 'is_murder', 'comment', 'comment_num', 'year', 'month']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in json_file:
        writer.writerow(i)

In [86]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

OverflowError: Python int too large to convert to C long

In [85]:
temp = pd.read_csv('조현병_20190801.csv', engine='python', encoding='UTF-8')
temp.shape

ParserError: field larger than field limit (131072)

In [72]:
for row in range(len(news)):
    news[row]['year'] = news[row]['date'].split('.')[0]
    news[row]['month'] = news[row]['date'].split('.')[1]

In [73]:
news[3]

{'title': '“안인득 강제 입원 시도했으나 좌절”…이유는?',
 'date': '2019.04.23. 오전 8:31',
 'content': '친절한 뉴스 두 번째 소식은 경남 진주에서 발생한 아파트 방화 살인 사건 피의자 안인득 관련 소식입니다.사건 발생 일주일이 돼 가지만 피해 유족과 주민들은 집 밖에 나가는 것이 무섭다며 극심한 후유증을 호소하고 있는데요.이번 사건 역시 전후 사정을 잘 살펴보면 사전에 막을 수 있었는데 하는 안타까움이 남습니다. 범죄 발생 2주 전쯤 안 씨의 가족들이 안 씨를 강제 입원시키려 했지만 제도의 벽에 가로막혀 번번이 무산된 정황이 드러났기 때문입니다.대체 뭐가 문제였을까요?현행법상 정신질환자를 강제 입원시킬 수 있는 제도로는 크게 3가지가 있습니다.보호자 2명과 전문의 2명이 신청하는 보호입원, 전문의나 경찰이 지자체에 요청하는 행정입원, 의사와 경찰관의 동의 아래 의뢰하는 응급입원입니다.하지만 안 씨는 이 세 가지를 모두 비켜 갔습니다. 먼저 보호입원과 행정입원은 안 씨가 병원에 가자는 형의 말을 거부하는 바람에 전문의 진단서를 받지 못해 뜻을 이룰 수가 없었습니다.응급입원도 안 씨가 자신의 잘못을 인정한다는 이유로 경찰이 위험 가능성이 높지 않다고 판단해 받아들여지지 않았습니다.들어 보시죠. [경찰 관계자/음성변조 : "강제입원을 시키거나 할 방법이 없느냐고 문의한 거예요. 자·타해 위험이 엄청 높을 때 (응급입원이) 가능하지 않습니까. (당시에) 그런 정도 자체가 아니었죠."]여기에 2017년 5월 개정된 정신건강복지법도 걸림돌이 됐습니다.환자의 인권을 강화한 게 강제 입원을 어렵게 만들었습니다.그렇게 2주가 흘렀고, 안 씨의 병세는 걷잡을 수 없이 악화되면서 이번 참극을 불렀습니다. 지난 연말이었죠?피의자 안인득처럼 정신 질환을 앓았던 사람에게 우리는 훌륭한 의사 한 명을 잃었던 기억이 있습니다.자신이 치료하던 환자가 휘두른 흉기에 숨진 고 임세원 교수입니다.당시 정부는 정신건강복지법을 다시 개정해 이른바 \'임

In [28]:
import json

with open('네이버뉴스_조현병_20190801.txt', 'r') as outfile:
    news = json.load(outfile)

In [17]:
import json

with open('네이버뉴스_조현병_total.txt','r') as f:
    l = json.load(f)

In [18]:
data = l[0]
data['title'], data['date'], data['content'], data['company'], data['url'], data['is_murder'], str(data['comment']), data['comment_num']

('조현병 10대 흉기 살해 뒤 태연히 집으로…',
 '2019.04.25. 오전 6:45',
 '\n\n\n\n동영상 뉴스\t\n\t[뉴스투데이]◀ 앵커 ▶안인득 사건의 충격이 가시기도 전에, 또다시 조현병을 앓고 있던 10대가 이웃 할머니를 살해한 사건이 일어났습니다. 할머니와 말다툼을 벌인 뒤 복도에 숨어 기다리다 흉기를 휘둘렀습니다.보도에 이재경 기자입니다.◀ 리포트 ▶한 여성이 들것에 실려 엘리베이터에서 나오고, 급하게 심폐소생술을 받은 뒤 구급차에 실립니다. 경남 창원의 한 아파트 6층 복도에서 74살 김모 할머니가 흉기에 찔렸습니다. 할머니는 병원으로 옮겨졌지만 결국 숨졌습니다. 피의자는 바로 아랫집에 사는 18살 A군.A군은 김 할머니 집에 찾아가 말다툼을 벌인 뒤, 복도에 숨어 기다리다, 외출을 위해 집 밖으로 나온 할머니에게 흉기를 휘둘렀습니다. 이후 A군은 태연하게 아파트를 나가 인근 미술관 화장실에서 손을 씻은 뒤 10분 만에 집으로 돌아왔습니다.A군은 범행을 저지른 뒤 아버지와 함께 집에 있다 목격자의 신고를 받고 출동한 경찰에 긴급 체포됐습니다.2년 전 고등학교를 자퇴한 A군은 지난해 정신병원에서 편집성 정신분열증 진단을 받은 사실이 확인됐습니다.경찰은 A군이 전에도 층간소음 문제로 숨진 할머니와 마찰을 빚었지만 범행의 직접적인 원인으로 보기는 어렵다며, 정신 이상에 무게를 두고 있다고 밝혔습니다.경찰은 정확한 범행 동기를 밝히기 위해 프로파일러를 투입해 수사하는 한편, A군에 대해 살인 혐의로 구속영장을 신청할 방침입니다.MBC뉴스 이재경입니다.이재경 기자 [저작권자(c) MBC (http://imnews.imbc.com) 무단복제-재배포 금지]▶ 네이버 홈에서 [MBC뉴스] 채널 구독하기▶ [엠빅뉴스] 고객님 화장실엔 가면 안 되나요? 화장실 차별에 직원들은 아파요▶ [14F] 의료사고, 대리수술 … 수술실 CCTV 설치가 답?Copyright(c) Since 1996, MBC&iMBC All rights reserved.\n\t\

In [21]:
import csv

with open('조현병.csv', 'w', encoding='utf-8', newline='') as f:
    fieldnames = ['title', 'date', 'content', 'company', 'url', 'is_murder', 'comment', 'comment_num']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in l:
        writer.writerow(i)

ValueError: dict contains fields not in fieldnames: 'link'

In [31]:
temp = pd.DataFrame(columns=l[0].keys())

for i, data in enumerate(l):
    temp.loc[i] = [data['title'], data['date'], data['content'], data['company'], data['url'], data['is_murder'], str(data['comment']), data['comment_num']]

KeyError: 'url'

In [23]:
temp.head()

,title,date,content,company,url,is_murder,comment,comment_num
0,"(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45...","(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45...","(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45...","(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45...","(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45...","(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45...","(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45...","(조현병 10대 흉기 살해 뒤 태연히 집으로…, 2019.04.25. 오전 6:45..."
1,"(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ...","(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ...","(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ...","(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ...","(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ...","(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ...","(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ...","(""할머니가 머릿속에 들어와""…조현병 10대 '흉기 살해', 2019.04.24. ..."
2,"([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019....","([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019....","([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019....","([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019....","([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019....","([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019....","([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019....","([김경래의 최강시사] 조현병환우회 “치료거부 환자에 공권력 행사해야”, 2019...."
3,"(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3...","(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3...","(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3...","(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3...","(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3...","(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3...","(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3...","(“안인득 강제 입원 시도했으나 좌절”…이유는?, 2019.04.23. 오전 8:3..."
4,"(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:...","(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:...","(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:...","(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:...","(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:...","(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:...","(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:...","(안인득, 조현병 치료 중단 후 33개월 '방치', 2019.04.22. 오전 9:..."


In [17]:
count = 0
list_ = []
for data in l:
    if data['content'] == '':
        print(data)
        l.remove(data)
        
print(count)
print(len(l))

{'title': '', 'date': '', 'content': '', 'company': '채널A', 'url': 'http://news.ichannela.com/society/3/03/20131023/58404245/1', 'is_murder': '0', 'comment': [], 'comment_num': 0}
{'title': '', 'date': '', 'content': '', 'company': 'KBS', 'url': 'http://news.kbs.co.kr/news/naverNewsView.do?SEARCH_NEWS_CODE=2667344', 'is_murder': '0', 'comment': [], 'comment_num': 0}
{'title': '', 'date': '', 'content': '', 'company': '채널A', 'url': 'http://news.ichannela.com/society/3/03/20130507/54955846/1', 'is_murder': '0', 'comment': [], 'comment_num': 0}
{'title': '', 'date': '', 'content': '', 'company': 'KBS', 'url': 'http://news.kbs.co.kr/news/NewsView.do?SEARCH_NEWS_CODE=2643554', 'is_murder': '0', 'comment': [], 'comment_num': 0}
{'title': '', 'date': '', 'content': '', 'company': '채널A', 'url': 'http://news.ichannela.com/society/3/03/20130119/52434848/1', 'is_murder': '0', 'comment': [], 'comment_num': 0}
{'title': '', 'date': '', 'content': '', 'company': 'KBS', 'url': 'http://news.kbs.co.kr/s

In [18]:
3309-23

3286

In [36]:
count = 0
list_ = []
for data in l:
    if data['content'] == '':
        print(data)
        count+=1
        
        l.remove(data)
print(count)

0


In [35]:
len(l)

3286

In [37]:
import json

with open('네이버뉴스_조현병_final.txt', 'w') as outfile:
    json.dump(l, outfile, default=str)

In [38]:
import json

with open('네이버뉴스_조현병_final.txt', 'r') as outfile:
    temp = json.load(outfile)
    print(len(temp))

3286


In [ ]:
notnull_df = df_content.loc[df_content['title'].isnull() == False].reset_index(drop=True)
notnull_df.head()

In [ ]:
comment_len = []
comment_cont = []
for i in range(len(notnull_df)):
    if notnull_df['url'].loc[i] != '':
        comment_list.append(len(get_comments(notnull_df['url'].loc[i])))
        comment_cont.append(get_comments(notnull_df['url'].loc[i]))
    else:
        comment_len.append('')
        comment_cont.append('')

In [ ]:
com_df = pd.DataFrame(columns=['title','date','content','company','url', 'comment_num', 'comment_cont'])
com_df = notnull_df
com_df['comment_num'] = comment_len
com_df['comment_cont'] = comment_cont
com_df.head()

In [ ]:
list = []
for i in com_df:
    dict_ = {'title':title, 'date':date, 'company':company, 'url':url, 'comment_num':comment, 'comment_cont':comment_cont}
    list.append(dict_)
    
json.dumps()

In [ ]:
df = pd.read_csv('네이버뉴스_0.csv', encoding='UTF-8', engine='python')
df1 = pd.read_csv('네이버뉴스_1.csv', encoding='UTF-8', engine='python')
df2 = pd.read_csv('네이버뉴스_2.csv', encoding='UTF-8', engine='python')

for i in tqdm(range(2,16)):
    date_ = []
    contents = []
    comments_len = []
    comments_cont = []
    temp = pd.read_csv('네이버뉴스_{}.csv'.format(i),encoding='UTF-8', engine='python')
    
    
    url = temp['url']
    for j in range(len(url)):
        url_ = url.loc[i]
        response = urllib.request.urlopen(url_)
        soup = BeautifulSoup(response, "html.parser")
        #    results = soup.select('div[class=ranking_headline]')  
        try: 
            btext = soup.find('div', {'id':'articleBodyContents'}).text.replace('\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n', '')
            #content = soup.select_one("#articleBodyContents").get_text.replace('\n'," ")
            #btext = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "") 
        except AttributeError:
            btext = ''
            print("content_error", i,j, url_)
        comments_len =  len(get_comments(url_))
        comments_cont = get_comments(url_)

        try:
            date = soup.select('span.t11')[0].text  
        except IndexError:
            date=''
            print("date_error",i,j, url_)

        #    time.sleep(0.1)
        temp0 = [date]
        temp1 = [btext.strip()]
        temp2 = [comment_len]
        date_.append(temp)
        contents.append(temp1)
        comments_len.append(temp2)
        comments_cont.append(comment_cont)
    temp = pd.DataFrame(temp)
    temp['date'] = date_
    temp['contents'] = contents
    temp['comments_len'] = comments_len
    temp['comments_cont'] = comments_cont
    df2 = pd.concat([df2, temp], sort=False)
#df2.to_csv('article_dataframe.csv',index=False)